In [44]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'cleanup.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [70]:
players <- read_csv("players.csv")


session <- read_csv("sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


After viewing the data sets I can see that the start_time and end_time in sessions is not in tidy data format and therefore will need to be wrangled before used in analysis however this not relevant to my question of interest. There is also no duration of play which would be beneficial for analysis but easily computed. Additionally, the time in which the players log on may be from different time zones and this could alter when they think the most users are online. 

In [73]:
#number of observations and variables 
player_obs_and_vars<- dim(players)


player_vars_name_and_type<- tibble(variable = names(players), 
                          type = map_chr(players, typeof), 
                          class = map_chr(players, ~class(.x)[1]))

player_obs_and_vars
player_vars_name_and_type

[1] 196   7

variable,type,class
<chr>,<chr>,<chr>
experience,character,character
subscribe,logical,logical
hashedEmail,character,character
⋮,⋮,⋮
name,character,character
gender,character,character
Age,double,numeric


The above code shows that there are 196 observations in the player data set and 7 variables. The data set was analyzed and oragnized into a tibble showing the variable names, the type of variable and the class of variable. We can see the the variables experience, hashedEmail, name, and gender all share the same type and class of character. While the age variable is of type double and class numeric and the suscribe variable is of class and type logical. 

In [74]:
#What the variables mean 
variable_meaning<- list(
experience=  "A character variable that tells us the what the players abilities are.",  
suscribe =  "Tells us if a player is suscribed to the game related newsletter.", 
hashedEmail = "String of characters that represents the players email", 
played_hours = "The number of hours played", 
name = "Tells us the username of the player.", 
gender= "Indicates the gender of the player.", 
Age= "Tells us the age of the player.") 
variable_meaning

$experience
[1] "A character variable that tells us the what the players abilities are."

$suscribe
[1] "Tells us if a player is suscribed to the game related newsletter."

$hashedEmail
[1] "String of characters that represents the players email"

$played_hours
[1] "The number of hours played"

$name
[1] "Tells us the username of the player."

$gender
[1] "Indicates the gender of the player."

$Age
[1] "Tells us the age of the player."

In [55]:
session_obs_and_vars<- dim(session)

session_vars_name_and_type<- tibble(variable = names(session), 
                                    type = map_chr(session, typeof), 
                                    class = map_chr(session, ~class(.x)[1]))
session_obs_and_vars
session_vars_name_and_type

[1] 1535    5

variable,type,class
<chr>,<chr>,<chr>
hashedEmail,character,character
start_time,character,character
end_time,character,character
original_start_time,double,numeric
original_end_time,double,numeric


The code above is the same as the first set of code but applied to the sessions data set. The data set has 1535 observations and 5 variables. The code organized the data variables into a tibble that shoes the type and class of each variable. The varaibles hashedEmail, start_time, and end_time are all of type and class character. While the variables original_start_time and original_end_time are of the type double and class numeric. 

In [57]:
#Sessions variables 
variables_meaning_sessions<- list(
    hashedEmail = "A string of characters that represents the players email", 
    start_time = "The date and time the session started.", 
    end_time = "The date and time the session ended.", 
    original_start_time = "The time the session started in the Unix timestamp format and is used for calculations.", 
    original_end_time = "The time the session ended in the Unix timestamp format and is used for calculations.")
variables_meaning_sessions

$hashedEmail
[1] "A string of characters that represents the players email"

$start_time
[1] "The date and time the session started."

$end_time
[1] "The date and time the session ended."

$original_start_time
[1] "The time the session started in the Unix timestamp format and is used for calculations."

$original_end_time
[1] "The time the session ended in the Unix timestamp format and is used for calculations."

In [48]:
mean_values_players<- players|> 
select(where(is.numeric))|>
summarise(across(everything(), mean, na.rm = TRUE))|>
pivot_longer(everything(),
               names_to = "Variable",
               values_to = "Mean")|> 
mutate(Mean= round(Mean, 2))

mean_values_players

Variable,Mean
<chr>,<dbl>
played_hours,5.85
Age,21.14


In [49]:
mean_values_sessions<- session|> 
select(where(is.numeric))|> 
summarise(across(everything(), mean, na.rm = TRUE))|> 
pivot_longer(everything(), names_to = "Variable", 
             values_to = "Mean")|> 
mutate(Mean = round(Mean, 2))

print(mean_values_sessions, digits = 2)

# A tibble: 2 × 2
  Variable               Mean
  <chr>                 <dbl>
1 original_start_time 1.72e12
2 original_end_time   1.72e12


Can player experience and/or played hours predict suscription to the game related newletter? 

The data in players.csv will help me address the question as it contains the variables player_hours, experience and suscribe. The suscribed variable is the response variable and gives a true or false categorical answer. While experience and played hours are the predictive varaibles.

By wrangling the data to see if there is a linear relationship between experience and suscription I can see if I should use a linear regression model or a knn-classification model. I can then apply this same process to played hours and can see which model to use. I can then also see if there is a linear relationship between experience and played hours and see if together these varaibles can be accurate predictors for knn-classification model. 

In order to address my question I will use predictive/classification models. I will begin by visualizing if there is a linear relationship between the predictive variables (experience and played_hours) and the response variable (suscribe). If there is a linear relationship I will then use linear regression to proceed with my prediction model. This method is appropriate as I am asking a predictive question where I am looking to see how a predictive variable is related to the response variable. A potential limitation to this model is that my variables are of different types and therefore using both played_hours and experience will be more challenging. It will not be possible to my knowledge to use a scatter plot to see if experience and suscribe have a linear relationship, but we can visualize it in bar graph to see if there is any relationship. Additionally, analysis of the  specificity and sensitivity may be challenging for me to interpret as I am not well versed in this genre of data. In order to train the model I will split the data into a 65/35 split and then use a 5 fold cross validation in order to optimize the selection of K. 

In [ ]:
#Visulization of experience vs suscription 
